# [PyTorch Tutorial](https://pytorch.org/tutorials/beginner/basics/intro.html)

In [18]:
import os
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor

## Load Data

In [19]:
training_data = datasets.FashionMNIST(
    root="../data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="../data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

## Build Model

References:

* [How to build a Model](https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html)
* [torch.nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html)

### Model Class

In [20]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "cpu"
)
print(f"Using {device} device")

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    # See https://pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module.forward
    # for when to call it and when not
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [21]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"{logits = }\n{logits.sum() = }\n{pred_probab = }\n{pred_probab.sum() = }\n{y_pred =}")

logits = tensor([[-0.0660,  0.0944, -0.0932,  0.0141,  0.0612, -0.0055,  0.0494,  0.0192,
         -0.0237,  0.0067]], grad_fn=<AddmmBackward0>)
logits.sum() = tensor(0.0567, grad_fn=<SumBackward0>)
pred_probab = tensor([[0.0929, 0.1091, 0.0905, 0.1007, 0.1056, 0.0987, 0.1043, 0.1012, 0.0970,
         0.1000]], grad_fn=<SoftmaxBackward0>)
pred_probab.sum() = tensor(1., grad_fn=<SumBackward0>)
y_pred =tensor([1])


### Model Layers

In [22]:
input_image = torch.rand(3,28,28)
print(input_image.size())
# print(input_image)

flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())
# print(flat_image)

layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())
# print(hidden1)

print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

torch.Size([3, 28, 28])
torch.Size([3, 784])
torch.Size([3, 20])
Before ReLU: tensor([[-0.0019,  0.1157, -0.7759, -0.1643,  0.2602,  0.2228,  0.1799,  0.2149,
          0.2800,  0.1379,  0.2853, -0.0402,  0.1322,  0.6066, -0.1114, -0.3746,
          0.1792, -0.0632,  0.1813,  0.0560],
        [-0.1496,  0.1236, -0.9442, -0.2122, -0.0850,  0.8782,  0.2493,  0.3142,
         -0.3503,  0.3442,  0.0559, -0.2049,  0.5206,  0.3009, -0.1630, -0.1066,
          0.3401, -0.2602,  0.0858,  0.2773],
        [ 0.2650,  0.1392, -0.7896, -0.1507,  0.3831,  0.2202,  0.1567,  0.2162,
         -0.1271,  0.2939,  0.5730,  0.1735,  0.2598,  0.6785, -0.0519, -0.1906,
          0.0692, -0.0741,  0.3304, -0.0486]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.1157, 0.0000, 0.0000, 0.2602, 0.2228, 0.1799, 0.2149, 0.2800,
         0.1379, 0.2853, 0.0000, 0.1322, 0.6066, 0.0000, 0.0000, 0.1792, 0.0000,
         0.1813, 0.0560],
        [0.0000, 0.1236, 0.0000, 0.0000, 0.0000, 0.8782, 0.2493, 0.31

In [23]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)
print(f"{logits = }\n{pred_probab = }")


logits = tensor([[-0.0114, -0.3132, -0.2463,  0.2200, -0.2106, -0.1449, -0.1778, -0.1577,
          0.1395,  0.0330],
        [-0.0775, -0.3573, -0.1425,  0.2955, -0.1293, -0.1023, -0.0412, -0.2331,
          0.1066, -0.0099],
        [ 0.0926, -0.2758, -0.1277,  0.1800, -0.1670, -0.1102, -0.0734, -0.1680,
          0.0116, -0.0865]], grad_fn=<AddmmBackward0>)
pred_probab = tensor([[0.1063, 0.0786, 0.0841, 0.1340, 0.0871, 0.0930, 0.0900, 0.0919, 0.1237,
         0.1112],
        [0.0977, 0.0739, 0.0916, 0.1419, 0.0928, 0.0953, 0.1013, 0.0836, 0.1175,
         0.1045],
        [0.1170, 0.0809, 0.0938, 0.1277, 0.0902, 0.0955, 0.0991, 0.0901, 0.1079,
         0.0978]], grad_fn=<SoftmaxBackward0>)


### Model Parameters

References

* [torch.nn.Module.named_parameters](https://pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module.named_parameters)
* [torch.nn.parameter.Parameter](https://pytorch.org/docs/stable/generated/torch.nn.parameter.Parameter.html#torch.nn.parameter.Parameter)

In [24]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 1.2308e-02, -8.7407e-03, -1.3371e-02,  ...,  1.6829e-02,
         -1.1037e-02, -3.5211e-03],
        [-1.9919e-02,  2.9085e-02,  8.6011e-03,  ..., -2.4071e-02,
          1.2363e-03,  8.1914e-06]], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0105, -0.0327], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0385,  0.0431,  0.0142,  ..., -0.0198, -0.0311, -0.0056],
        [-0.0160,  0.0120,  0.0422,  ..., -0.0197, -0.0243,  0.0439]],
       gra

## Train Model

References

* [How to train a model](https://pytorch.org/tutorials/beginner/basics/optimization_tutorial.html)
* [torch.nn.CrossEntropyLoss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html)

In [25]:
learning_rate = 1e-3

# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward() # NOTE: Could/Should it be loss.sum().backward()?
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            # NOTE: Here pred is not mapped to probablities before argmax. It's an optimazation.
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

    

In [27]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 0.817064  [   64/60000]
loss: 0.883475  [ 6464/60000]
loss: 0.655078  [12864/60000]
loss: 0.854801  [19264/60000]
loss: 0.759019  [25664/60000]
loss: 0.746451  [32064/60000]
loss: 0.825318  [38464/60000]
loss: 0.790091  [44864/60000]
loss: 0.791187  [51264/60000]
loss: 0.759871  [57664/60000]
Test Error: 
 Accuracy: 71.9%, Avg loss: 0.771097 

Epoch 2
-------------------------------
loss: 0.773234  [   64/60000]
loss: 0.851362  [ 6464/60000]
loss: 0.618294  [12864/60000]
loss: 0.827294  [19264/60000]
loss: 0.734918  [25664/60000]
loss: 0.717412  [32064/60000]
loss: 0.798285  [38464/60000]
loss: 0.769787  [44864/60000]
loss: 0.764377  [51264/60000]
loss: 0.734872  [57664/60000]
Test Error: 
 Accuracy: 73.2%, Avg loss: 0.745893 

Epoch 3
-------------------------------
loss: 0.735958  [   64/60000]
loss: 0.822884  [ 6464/60000]
loss: 0.587413  [12864/60000]
loss: 0.804657  [19264/60000]
loss: 0.714459  [25664/60000]
loss: 0.693958  [32064/600

## Save and Load Model

In [31]:
model_path = "../model/pytorch-tut"

torch.save(model, model_path)

model = torch.load(model_path)
print(model)

# Validate the loaded model
test_loop(test_dataloader, model, loss_fn)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)
Test Error: 
 Accuracy: 75.7%, Avg loss: 0.687445 

